In [1]:
import pandas as pd
import numpy as np 

In [6]:
df=pd.read_csv('../datasets/fake-news/train.csv')

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df=df.dropna()

In [9]:
 X=df.drop('label',axis=1)

In [10]:
y=df['label']

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.12.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
messages=X.copy()
messages.reset_index(inplace=True)

In [26]:
voc_size=10000

In [16]:
import nltk
import re #regular expression
from nltk.corpus import stopwords

In [23]:
# dataset preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    #print(i)
    review=re.sub('[^a-zA-Z]',' ',messages['title'][i]) #substiting everything with blank except a-z and A- Z
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review) 

In [28]:
#corpus

In [29]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
#onehot_repr

In [31]:
# Padding
sent_size=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_size)
print(embedded_docs)

[[   0    0    0 ... 7593 3078  678]
 [   0    0    0 ... 8343 8753 7283]
 [   0    0    0 ... 1460  681 8758]
 ...
 [   0    0    0 ... 9762 2501 1154]
 [   0    0    0 ... 7079 6077 5782]
 [   0    0    0 ... 1176 2039 4537]]


In [32]:
len(embedded_docs)

18285

In [33]:
len(corpus)

18285

In [34]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_size))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            400000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
y.shape

(18285,)

In [36]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [38]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [49]:
# Adding Dropout

from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_size))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            400000    
                                                                 
 dropout_2 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [50]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [51]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 34ms/step - loss: 0.3382 - accuracy: 0.8436 - val_loss: 0.1914 - val_accuracy: 0.9162
Epoch 2/10
192/192 [==============================] - 6s 34ms/step - loss: 0.1419 - accuracy: 0.9450 - val_loss: 0.1861 - val_accuracy: 0.9244
Epoch 3/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0959 - accuracy: 0.9667 - val_loss: 0.2051 - val_accuracy: 0.9241
Epoch 4/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0687 - accuracy: 0.9748 - val_loss: 0.2711 - val_accuracy: 0.9178
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0504 - accuracy: 0.9831 - val_loss: 0.2706 - val_accuracy: 0.9178
Epoch 6/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0373 - accuracy: 0.9879 - val_loss: 0.3118 - val_accuracy: 0.9171
Epoch 7/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0310 - accuracy: 0.9889 - val_loss: 0.3155 - val_accuracy: 0.9165

In [63]:
# performance metrics and accuracy
y_pred=(model.predict(X_test) > 0.5).astype("int32")
y_pred

189/189 [==============================] - 1s 5ms/step


array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [64]:
from sklearn.metrics import confusion_matrix

In [65]:
confusion_matrix(y_test,y_pred)

array([[3133,  286],
       [ 241, 2375]], dtype=int64)

In [66]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9126760563380282